Обучение EfficientNet

In [ ]:
import os
import json
import torch
import torchvision
from torchvision import transforms
from torch.utils.data import Dataset, DataLoader
from PIL import Image
from efficientnet_pytorch import EfficientNet
from torch import nn, optim
from tqdm import tqdm

In [ ]:
class MedicalPPEDataset(Dataset):
    def __init__(self, coco_annotation_file, image_dir, transform=None):
        with open(coco_annotation_file, 'r') as f:
            self.coco_data = json.load(f)
        self.image_dir = image_dir
        self.transform = transform
        
        # Create a mapping of image_id to file_name
        self.image_id_to_file = {img['id']: img['file_name'] for img in self.coco_data['images']}
        
        # Create a mapping of image_id to category_id (assuming single class per image)
        self.image_id_to_category = {}
        for ann in self.coco_data['annotations']:
            self.image_id_to_category[ann['image_id']] = ann['category_id']
        
        self.categories = self.coco_data['categories']
        self.num_classes = len(self.categories)

    def __len__(self):
        return len(self.coco_data['images'])

    def __getitem__(self, idx):
        img_info = self.coco_data['images'][idx]
        img_path = os.path.join(self.image_dir, img_info['file_name'])
        image = Image.open(img_path).convert('RGB')
        
        if self.transform:
            image = self.transform(image)
        
        label = self.image_id_to_category[img_info['id']]
        return image, label

In [ ]:
# Set up data transforms
data_transforms = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

In [ ]:
# Create datasets
train_dataset = MedicalPPEDataset('path/to/train_annotations.json', 'path/to/train_images', transform=data_transforms)
val_dataset = MedicalPPEDataset('path/to/val_annotations.json', 'path/to/val_images', transform=data_transforms)

In [ ]:
# Create data loaders
train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True, num_workers=4)
val_loader = DataLoader(val_dataset, batch_size=16, shuffle=False, num_workers=4)

In [ ]:
# Set up the model
model = EfficientNet.from_pretrained('efficientnet-b0', num_classes=train_dataset.num_classes)

In [ ]:
# Set up loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

In [ ]:
# Training function
def train_model(model, criterion, optimizer, num_epochs=10):
    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
    model.to(device)

    for epoch in range(num_epochs):
        print(f'Epoch {epoch+1}/{num_epochs}')
        print('-' * 10)

        model.train()
        running_loss = 0.0
        running_corrects = 0

        for inputs, labels in tqdm(train_loader):
            inputs = inputs.to(device)
            labels = labels.to(device)

            optimizer.zero_grad()

            outputs = model(inputs)
            _, preds = torch.max(outputs, 1)
            loss = criterion(outputs, labels)

            loss.backward()
            optimizer.step()

            running_loss += loss.item() * inputs.size(0)
            running_corrects += torch.sum(preds == labels.data)

        epoch_loss = running_loss / len(train_dataset)
        epoch_acc = running_corrects.double() / len(train_dataset)

        print(f'Train Loss: {epoch_loss:.4f} Acc: {epoch_acc:.4f}')

        # Validation phase
        model.eval()
        running_loss = 0.0
        running_corrects = 0

        with torch.no_grad():
            for inputs, labels in tqdm(val_loader):
                inputs = inputs.to(device)
                labels = labels.to(device)

                outputs = model(inputs)
                _, preds = torch.max(outputs, 1)
                loss = criterion(outputs, labels)

                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds == labels.data)

        epoch_loss = running_loss / len(val_dataset)
        epoch_acc = running_corrects.double() / len(val_dataset)

        print(f'Val Loss: {epoch_loss:.4f} Acc: {epoch_acc:.4f}')

    return model

In [ ]:
# Train the model
trained_model = train_model(model, criterion, optimizer, num_epochs=10)

# Save the trained model
torch.save(trained_model.state_dict(), 'medical_ppe_efficientnet.pth')

# Function to test the model on a single image
def test_single_image(model, image_path):
    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
    model.to(device)
    model.eval()

    image = Image.open(image_path).convert('RGB')
    image = data_transforms(image).unsqueeze(0).to(device)

    with torch.no_grad():
        outputs = model(image)
        _, preds = torch.max(outputs, 1)

    return train_dataset.categories[preds.item()]['name']

# Example usage of the test function
test_image_path = 'path/to/test_image.jpg'
predicted_class = test_single_image(trained_model, test_image_path)
print(f'Predicted class: {predicted_class}')

-----

In [6]:
import os
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from torchvision import transforms
from efficientnet_pytorch import EfficientNet
from PIL import Image
import numpy as np

In [7]:
class YOLODataset(torch.utils.data.Dataset):
    def __init__(self, data_dir, transform=None):
        self.data_dir = data_dir
        self.transform = transform
        self.image_files = [f for f in os.listdir(data_dir) if f.endswith('.jpg') or f.endswith('.png')]
        
        if len(self.image_files) == 0:
            raise ValueError(f"No image files found in {data_dir}")

    def __len__(self):
        return len(self.image_files)

    def __getitem__(self, idx):
        img_path = os.path.join(self.data_dir, self.image_files[idx])
        label_path = os.path.splitext(img_path)[0] + '.txt'

        if not os.path.exists(img_path):
            raise FileNotFoundError(f"Image file not found: {img_path}")
        if not os.path.exists(label_path):
            raise FileNotFoundError(f"Label file not found: {label_path}")

        image = Image.open(img_path).convert('RGB')
        
        if self.transform:
            image = self.transform(image)

        with open(label_path, 'r') as f:
            labels = []
            for line in f.readlines():
                class_id, x, y, w, h = map(float, line.strip().split())
                labels.append([class_id, x, y, w, h])

        labels = torch.tensor(labels)
        return image, labels

# Set up data transforms
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

# Create dataset and data loader
data_dir = 'D://Work//coding//vuzz//new//MIREA_7_semester_VT//rsppr//SAFE-MACS//data//cppe5//train'
try:
    train_dataset = YOLODataset(data_dir, transform=transform)
    train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True, num_workers=4)
    print(f"Dataset size: {len(train_dataset)}")
except Exception as e:
    print(f"Error creating dataset: {str(e)}")
    exit(1)

# Initialize EfficientNet model
num_classes = 5  # Set this to the number of classes in your dataset
model = EfficientNet.from_pretrained('efficientnet-b7', num_classes=num_classes)

# Define loss function and optimizer
criterion = nn.MSELoss()  # You may need to customize this based on your specific requirements
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Training loop
num_epochs = 50
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)

for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0

    for images, labels in train_loader:
        images = images.to(device)
        labels = labels.to(device)

        optimizer.zero_grad()

        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

    epoch_loss = running_loss / len(train_loader)
    print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {epoch_loss:.4f}")

# Save the trained model
torch.save(model.state_dict(), 'efficientnet_ppe_model.pth')

Error creating dataset: No image files found in D://Work//coding//vuzz//new//MIREA_7_semester_VT//rsppr//SAFE-MACS//data//cppe5//train


Downloading: "https://github.com/lukemelas/EfficientNet-PyTorch/releases/download/1.0/efficientnet-b7-dcc49843.pth" to C:\Users\user/.cache\torch\hub\checkpoints\efficientnet-b7-dcc49843.pth
  6%|▌         | 14.1M/254M [00:30<08:34, 490kB/s]


KeyboardInterrupt: 